In [10]:
!pip install accelerate -U
!pip install transformers[torch]

In [11]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

# example dataset
data = {
    'Sentence': [
        'Tires were delivered on time.',
        'The customer service was excellent.',
        'The product quality was poor.',
        'The website is user-friendly.',
        'The prices were competitive.',
        'The delivery was late but the customer service was excellent.',
        'The product quality was poor and the prices were high.'
    ],
    'Labels': [
        ['Delivery'],
        ['Customer Service'],
        ['Product Quality'],
        ['Website'],
        ['Pricing'],
        ['Delivery', 'Customer Service'],
        ['Product Quality', 'Pricing']
    ],
    'Sentiments': [
        ['Positive'],
        ['Positive'],
        ['Negative'],
        ['Positive'],
        ['Positive'],
        ['Negative', 'Positive'],
        ['Negative', 'Negative']
    ]
}

df = pd.DataFrame(data)

# tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = tokenizer(df['Sentence'].tolist(), padding=True, truncation=True, return_tensors='pt')

# encode labels and sentiments
labels = df['Labels'].tolist()
sentiments = df['Sentiments'].tolist()

all_labels = list(set([label for sublist in labels for label in sublist]))
all_sentiments = ['Positive', 'Negative']

label_encoder = MultiLabelBinarizer(classes=all_labels)
sentiment_encoder = MultiLabelBinarizer(classes=all_sentiments)

binary_labels = label_encoder.fit_transform(labels)
binary_sentiments = sentiment_encoder.fit_transform(sentiments)

# combine
combined_labels = np.hstack((binary_labels, binary_sentiments))

class CustomDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = sentences
        self.labels = labels

    def __len__(self):
        return len(self.sentences['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.sentences['input_ids'][idx],
            'attention_mask': self.sentences['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# dataset
dataset = CustomDataset(tokenized_texts, combined_labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=combined_labels.shape[1])

training_args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

predictions = trainer.predict(val_dataset)
predicted_labels = torch.sigmoid(torch.tensor(predictions.predictions)) > 0.5

# separate true labels and sentiments
true_labels = combined_labels[val_dataset.indices]
true_labels_only = true_labels[:, :len(all_labels)]
true_sentiments_only = true_labels[:, len(all_labels):]

# separate predicted labels and sentiments
predicted_labels_only = predicted_labels[:, :len(all_labels)].int().tolist()
predicted_sentiments_only = predicted_labels[:, len(all_labels):].int().tolist()

# flatten the lists for classification report
true_labels_flat = [item for sublist in true_labels_only.tolist() for item in sublist]
predicted_labels_flat = [item for sublist in predicted_labels_only for item in sublist]

true_sentiments_flat = [item for sublist in true_sentiments_only.tolist() for item in sublist]
predicted_sentiments_flat = [item for sublist in predicted_sentiments_only for item in sublist]

# output classification report for labels
print("Classification Report for Labels:\n", classification_report(true_labels_flat, predicted_labels_flat, target_names=all_labels, labels=[0, 1]))

# output classification report for sentiments
print("Classification Report for Sentiments:\n", classification_report(true_sentiments_flat, predicted_sentiments_flat, target_names=all_sentiments, labels=[0, 1]))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.654919
2,No log,0.631099
3,No log,0.629223


Classification Report for Labels:
                   precision    recall  f1-score   support

         Pricing       0.70      1.00      0.82         7
        Delivery       0.00      0.00      0.00         3

        accuracy                           0.70        10
       macro avg       0.35      0.50      0.41        10
    weighted avg       0.49      0.70      0.58        10

Classification Report for Sentiments:
               precision    recall  f1-score   support

    Positive       0.00      0.00      0.00         1
    Negative       0.75      1.00      0.86         3

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2326: UserWarning: labels size, 2, does not match size of target_names, 5
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero